<a href="https://colab.research.google.com/github/canerskrc/Borusan_Kod_Adi_Dijital/blob/main/borusan_day14_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Çalışma Prensibi

- Birden fazla Decision Tree'nin bir araya gelerek oluşturduğu algoritmadır.

- Her ağaç oluşturulurken tüm özelliklere aynı anda bakılmaz. Verilerin nasıl bölüneceğine karar vermek için rastgele birkaç özellik seçilir.

- Sınıflandırma için ağaçların çoğunun hemfikir olması yani çoğunluk oyu son sınıflandırma cevabını üretir. Regresyon için de son cevap ağaçların tahminlerinin ortalaması olduğundan bir sayı elde edilir.



In [ ]:
import numpy as np
import pandas as pd



In [ ]:
class Node:
  def __init__(self, feature=None, threshold=None, left = None, right = None, value= None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

In [ ]:
def entropy(y):
  values, counts = np.unique(y,  return_counts=True)
  probs = counts / counts.sum()
  return -np.sum(probs * np.log2(probs))

def gini(y):
  values, counts = np.unique(y,  return_counts=True)
  probs = counts / counts.sum()
  return -np.sum(probs ** 2)

def best_split(X,y,criterion="gini", feature_subset=None):
  best_feature, best_threshold, best_impurity = None, None, float("inf")
  impurity_func = gini if criterion == "gini" else entropy

  features = feature_subset if feature_subset is not None else X.colums

  for feature in features:
    unique_values = X[feature].unique() # her bir özellik için o özelliğin alabileceği tüm unique değerleri alır.

    for val in unique_values:

      if np.issubdtype(X[feature].dtype,np.number):
        left_mask = (X[feature] <= val)
      else:
        left_mask = (X[feature] == val)
      right_mask = ~left_mask #sol tarafta olmayan veriler sağ tarafa gider.

      if left_mask.sum() == 0 or right_mask.sum() == 0:
        continue

#Sağ ve sol impurity hesaplama

      left_impurity = impurity_func(y[left_mask])
      right_impurity = impurity_func(y[right_mask])

#Ağırlıklı ortalama impurity. Split'in toplam belirsizlik maliyetini ölçer.

      weighted = ((left_mask.sum()/len(y)) * left_impurity +
                           (right_mask.sum()/len(y)) * right_impurity)

#En iyi split'i güncelleme

      if weighted < best_impurity:
        best_feature = feature
        best_threshold = val
        best_impurity = weighted
  return best_feature, best_threshold, best_impurity


#Decision Tree Kurgusu

In [ ]:
def build_tree(X,y, depth=0, max_depth=5, criterion="gini", max_features=None):

  #Durdurma koşulu ( leaf oluşturma )

  if len(np.unique(y)) == 1 or depth >= max_depth:
    values, counts = np.unique(y, return_counts=True)
    return Node(value=values[np.argmax(counts)])
#Random özellik seçimi
  if max_features is None:
    features = X.columns
  else:
    features = np.random.choice(X.columns, max_features, replace=False )

#En iyi split'i bul

  feature, threshold = best_split(X,y, criterion, feature_subset=features) # hangi sütunu hangi eşikle bölelim? cevabı burada verilir.
  if feature is None:
    values,counts = np.unique(y, return_counts=True)
    return Node(value=values[np.argmax(counts)])

#Bölme maskeleri oluştur

  if np.issubdtype(X[feature].dtype, np.number):
    left_mask = (X[feature] <= threshold)
  else:
    left_mask = (X[feature] == threshold)
  right_mask = ~left_mask

#recursive çağrı(alt ağaçlar oluşturulur)

  left = build_tree(X[left_mask],y[left_mask], depth+1, max_depth, criterion)
  right = build_tree(X[right_mask],y[right_mask], depth+1, max_depth, criterion)

  return Node(feature, threshold, left,right)

#Random Forest

In [ ]:
from collections import Counter
class RandomForest:
  def __init__(self, n_trees = 10, max_depth = 5, criterion="gini", max_features=None):
    self.n_trees= n_trees
    self.max_depth = max_depth
    self.criterion = criterion
    self.max_features = max_features
    self.trees = []

  def fit(self, X,y):
    for _ in range(self.n_trees):
      idxs = np.random.choice(len(X),len(y),replace=True)
      X_sample, y_sample = X.iloc[idxs], y.iloc[idxs]

      tree = build_tree(
          X_sample, y_sample,
          max_depth = self.max_depth,
          criterion= self.criterion,
          max_features= self.max_features
      )
      self.trees.append(tree)

  def predict_one(self, x, tree):
    node = tree
    while node.value is None:
      if np.issubdtype(type(x[node.feature]), np.number):
        if x[node.feature] <= node.threshold:
          node = node.left
        else:
          node = node.right
      else:
        if x[node.feature] == node.threshold:
          node = node.left
        else:
          node = node.right
    return node.value

#Majority Voting

  def predict(self, X):
    tree_preds = []
    for tree in self.trees:
      preds = [self.predict_one(x,tree) for _, x in X.iterrows()]
      tree_preds.append(preds)

    tree_preds = np.array(tree_preds).T
    final_preds = [Counter(row).most_common(1)[0][0] for row in tree_preds]
    return final_preds




In [6]:
features = [
    "Curricular units 2nd sem (grade)",
    "Curricular units 1st sem (grade)",
    "Previous qualification (grade)",
    "Admission grade",
    "Age at enrollment"]

X_small = X[features]
y_small = y

rf = RandomForest(n_trees=5, max_depth=4, max_features=3)
rf.fit(X_small,y_small)
y_pred = rf.predict(X_small[:10])
print(y_pred)

NameError: name 'X' is not defined

#RandomForest HPO

- ağaç sayısı(n_estimators) modelin varyansı azalır. Big data ortamında 200-400 arasında değişir. Edge deployment için 50-100. Early stopping benzeri Out of bag hata grafiğiyle optimum ağaç sayısı belirlenir.

- Max_dept çok derin ağaçlar ezber yapar.(overfit). Tabular verilerde 10-20 arasında tutulur. max_depth=None olduğunda model tüm düğümleri açar. Bu da overfit riskini yükseltir.

- max_features düşük olursa varyans azalır ancak bias artar. yüksek olursa ağaçlar birbirine çok benzer kurgulanır ve korelasyon artar.

- bootstrap sampling( bootstrap, max_samples) bootstrap=false olduğunda her ağaç tüm detayı görür bu da bagging etkisini azaltır. max_samples ağaç başına veri sayısını ayarlar. Çok büyük veri setlerinde(100M+ satır) max_samples=0.3 ile training süresi ikonik biçimde azaltılır ama accuracy büyük ölçüde korunur.

- Out of Bag Error

#HPO Stratejileri

- Grid Search = Küçük datasetlerde klasik.

- random search = 10+ hiperparametre varsa daha verimli.

- Bayes optimizasyon = Akıllı arama tekniği

- Population Based Training ( google vizier, Uber michelangelo platformları bunu kullanır )
